In [62]:
import open3d as o3d
import numpy as np
import multiprocessing as mp
from multiprocessing import Pool
import copy as cp
import open3d.core as o3c
import matplotlib.pyplot as plt
import pyransac3d as pyrsc
import time

from scipy.spatial.transform import Rotation
from iteration_utilities import deepflatten
from mpl_toolkits.mplot3d import Axes3D

In [63]:
#load pcd file, filter, downsample
pcd = o3d.io.read_point_cloud("final_cropped.pcd")
pcd_flat = o3d.io.read_point_cloud("pcd_flat.pcd")
pcd_flat.paint_uniform_color([0,1,0])
pcd.estimate_normals()
#pcd.orient_normals_consistent_tangent_plane(40)
o3d.visualization.draw_geometries([pcd_flat])


In [64]:
def detect_ground_plane(pcd):
    plane_model, inliers = pcd.segment_plane(distance_threshold=0.01,
                                         ransac_n=3,
                                         num_iterations=1000)
    
    return plane_model, inliers
    
    

In [65]:
def compute_plane_transformation(plane1, plane2):
    """
    Computes the transformation between two plane equations of the form ax + by + cz + d = 0.

    Args:
        plane1 (tuple): Coefficients of the first plane equation (a1, b1, c1, d1).
        plane2 (tuple): Coefficients of the second plane equation (a2, b2, c2, d2).

    Returns:
        np.ndarray: A 4x4 transformation matrix.
    """
    # Extract coefficients of the first plane equation
    a1, b1, c1, d1 = plane1

    # Extract coefficients of the second plane equation
    a2, b2, c2, d2 = plane2

    # Compute the transformation matrix coefficients
    a = a2 * a1 + b2 * b1 + c2 * c1
    b = a2 * d1 - a1 * d2
    c = b2 * d1 - b1 * d2
    d = c2 * d1 - c1 * d2

    transformation_matrix = np.array([[a, 0, 0, b],
                                      [0, a, 0, c],
                                      [0, 0, a, d],
                                      [0, 0, 0, 1]])

    return transformation_matrix

In [66]:
def calculate_floor_alignment_matrix(pcd_ground_plane):
    a, b, c, d = pcd_ground_plane
    d = -d
    # Calculate the normal vector of the plane
    plane_normal = np.array([a, b, c])

    # Find the z-axis (floor normal) of the transformed coordinate system
    z_axis = plane_normal / np.linalg.norm(plane_normal)

    # Define the x-axis (assuming it points in the direction of the original x-axis)
    x_axis = np.array([1, 0, 0])

    # Calculate the y-axis (cross product of z_axis and x_axis)
    y_axis = np.cross(z_axis, x_axis)
    y_axis /= np.linalg.norm(y_axis)

    # Calculate the new x-axis (cross product of y_axis and z_axis)
    x_axis = np.cross(y_axis, z_axis)
    x_axis /= np.linalg.norm(x_axis)

    # Construct the transformation matrix
    transform_matrix = np.eye(4)
    transform_matrix[0, :3] = x_axis
    transform_matrix[1, :3] = y_axis
    transform_matrix[2, :3] = z_axis

    # Find the translation component to align the point cloud with the plane
    centroid_on_plane = -plane_normal * (d / np.linalg.norm(plane_normal))
    transform_matrix[:3, 3] = centroid_on_plane

    return transform_matrix

In [67]:
ground_plane = np.asarray([0,0,1,0])
pcd_ground_plane, inliers = detect_ground_plane(pcd)
inlier_cloud = pcd.select_by_index(inliers)
inlier_cloud.paint_uniform_color([1.0, 0, 0])

#trans = compute_plane_transformation(ground_plane, pcd_ground_plane)
trans = calculate_floor_alignment_matrix(pcd_ground_plane)
print(trans)
test = cp.deepcopy(pcd)
pcd_t = test.transform(trans)

o3d.visualization.draw_geometries([pcd_t, pcd_flat])

[[ 9.99558991e-01  7.41309839e-04  2.96862533e-02 -1.49097932e-02]
 [ 0.00000000e+00  9.99688358e-01 -2.49637031e-02  1.25284777e-02]
 [-2.96955077e-02  2.49526939e-02  9.99247487e-01  5.01711355e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [68]:
#o3d.io.write_point_cloud("corridor_ground_align.pcd", pcd_t)

In [69]:
bb = pcd_t.get_axis_aligned_bounding_box()
min_bound = bb.get_min_bound()
max_bound = bb.get_max_bound()

min_bound[0] = -47
min_bound[1] = -3
min_bound[2] = - 2

max_bound[0] = 3
#max_bound[1] =
max_bound[2] = 3
print(min_bound, max_bound)

pcd_new = cp.deepcopy(pcd_t)
crob_bb = o3d.geometry.AxisAlignedBoundingBox(min_bound, max_bound)
pcd_new = pcd_new.crop(crob_bb)


o3d.visualization.draw_geometries([pcd_new, pcd_flat])

[-47.  -3.  -2.] [ 3.         10.16708183  3.        ]


In [70]:
o3d.io.write_point_cloud("final_cropped_ground_align2.pcd", pcd_t)

True